# Example notebook using Qililab

In [6]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from qibo.gates import CZ, M
from qibo.models.circuit import Circuit

import qililab as ql

%matplotlib widget

In [7]:
fname = os.path.abspath("")
os.environ["RUNCARDS"] = str(Path(fname) / "runcards")
os.environ["DATA"] = str(Path(fname) / "data")

## Load a platform

In [8]:
runcard_path = str(Path(fname) / "runcards/galadriel.yml")
platform = ql.build_platform(runcard=runcard_path)

[qililab] [0.20.4|WARNING|2023-08-22 14:52:03]: Cannot set `output_i=1` and `output_q=0` in hardware. The I/Q signals sent to sequencer 1 will be swapped to allow this setting.
[qililab] [0.20.4|WARNING|2023-08-22 14:52:03]: Cannot set `output_i=3` and `output_q=2` in hardware. The I/Q signals sent to sequencer 3 will be swapped to allow this setting.


### Platform Chip

In [9]:
print(platform.chip)

Chip with 5 qubits and 12 ports: 

 * Port drive_line_q0 (drive): ----|qubit_0|----
 * Port drive_line_q1 (drive): ----|qubit_1|----
 * Port drive_line_q2 (drive): ----|qubit_2|----
 * Port drive_line_q3 (drive): ----|qubit_3|----
 * Port drive_line_q4 (drive): ----|qubit_4|----
 * Port flux_line_q0 (flux): ----|qubit_0|----
 * Port flux_line_q1 (flux): ----|qubit_1|----
 * Port flux_line_q2 (flux): ----|qubit_2|----
 * Port flux_line_q3 (flux): ----|qubit_3|----
 * Port flux_line_q4 (flux): ----|qubit_4|----
 * Port feedline_input (feedline_input): ----|resonator_q0|--|resonator_q1|--|resonator_q2|--|resonator_q3|--|resonator_q4|----
 * Port feedline_output (feedline_output): ----|resonator_q0|--|resonator_q1|--|resonator_q2|--|resonator_q3|--|resonator_q4|----



### Platform Buses

In [10]:
print(platform.buses)

Bus feedline_bus:  -----|QRM1|--|rs_1|------|resonator_q0|------|resonator_q1|------|resonator_q2|------|resonator_q3|------|resonator_q4|----
Bus drive_line_q0_bus:  -----|QCM-RF1|------|qubit_0|----
Bus flux_line_q0_bus:  -----|QCM1|------|qubit_0|----
Bus drive_line_q1_bus:  -----|QCM-RF1|------|qubit_1|----
Bus flux_line_q1_bus:  -----|QCM1|------|qubit_1|----
Bus drive_line_q2_bus:  -----|QCM-RF2|------|qubit_2|----
Bus flux_line_q2_bus:  -----|QCM2|------|qubit_2|----
Bus drive_line_q3_bus:  -----|QCM-RF3|------|qubit_3|----
Bus flux_line_q3_bus:  -----|QCM1|------|qubit_3|----
Bus drive_line_q4_bus:  -----|QCM-RF3|------|qubit_4|----
Bus flux_line_q4_bus:  -----|QCM1|------|qubit_4|----


## Connect to a Platform

In [ ]:
# Connect to all instruments of the platform and block the connection for other users
platform.connect(manual_override=False)  # if manual_override=True, it surpasses any blocked connection
platform.initial_setup()  # Sets all the values of the Runcard to the connected instruments
platform.turn_on_instruments()  # Turns on all instruments

## Create an experiment (Rabi example)

### Create the circuit associated with the experiment

In [11]:
qubit = 0  # Qubit to run the Rabi on
M_BUFFER_TIME = 0

# Define Circuit to execute
circuit = Circuit(qubit + 1)
circuit.add(ql.Drag(qubit, theta=np.pi, phase=0))
circuit.add(ql.Wait(qubit, M_BUFFER_TIME))
circuit.add(M(qubit))

MeasurementResult(qubits=(0,), nshots=0)

## Run the experiment

In [ ]:
results = []
sweep_interval = np.linspace(0.05, 0.35, num=61)
HW_AVG = 1000
REPETITION_DURATION = 200_000

# Run experiment
for amp in sweep_interval:
    platform.set_parameter(alias=f"Drag({qubit})", parameter=ql.Parameter.AMPLITUDE, value=float(amp))
    result = platform.execute(program=circuit, num_avg=HW_AVG, repetition_duration=REPETITION_DURATION)
    results.append(result.array)

results = np.hstack(results)

In [ ]:
job_name = f"AC-Ra-q{qubit}"
loops = {f"Drag({qubit})_amp": sweep_interval}
ql.save_results(results=results, loops=loops, data_path="../data/", name=job_name)

In [ ]:
def plot_iq(xdata, results: np.ndarray, title_label: str, xlabel: str):
    fig, axes = plt.subplots(1, 2, figsize=(13, 7))
    axes[0].plot(xdata, results[0], "--o", color="blue")
    axes[1].plot(xdata, results[1], "--o", color="blue")
    axes[0].set_title("I")
    axes[1].set_title("Q")
    axes[0].set_xlabel(xlabel)
    axes[1].set_xlabel(xlabel)
    axes[0].set_ylabel("Voltage [a.u.]")
    axes[1].set_ylabel("Voltage [a.u.]")
    fig.suptitle(title_label)
    return fig, axes


fig, axes = plot_iq(xdata=sweep_interval, results=results, title_label=f"Rabi q{qubit}", xlabel="Amplitude")
fig.show()

## Disconnect from a Platform

- Disconnect all instruments specified in the loaded platform
- Turns off the instruments only when explicitly specified (by default is set to False)

In [ ]:
platform.disconnect()

[qibo-connection] 0.7.2|INFO|2023-03-23 12:44:08]: Device Galadriel Qblox rack released.
INFO:qiboconnection.config:Device Galadriel Qblox rack released.
[qililab] [0.16.1|INFO|2023-03-23 12:44:08]: Already disconnected from the instruments
INFO:qililab.config.config:Already disconnected from the instruments
